In [2]:
# !pip install python-docx

In [14]:
from docx import Document
import pandas as pd
import re

def is_probable_title(text):
    # Eğer 1-6 kelime varsa ve her kelimenin baş harfi büyükse, başlık olabilir
    words = text.split()
    return (
        1 <= len(words) <= 6 and
        all(word[0].isupper() for word in words if word)
    )

def parse_fikralar(path):
    doc = Document(path)
    fıkralar = []
    current_title = None
    current_content = []
    id_counter = 1

    for para in doc.paragraphs:
        text = para.text.strip()
        if not text:
            continue

        # Format 1: 1. Başlık
        match = re.match(r"^(\d+)\.\s+(.*)", text)
        if match:
            if current_title and current_content:
                fıkralar.append({
                    "ID": id_counter,
                    "Başlık": current_title,
                    "İçerik": " ".join(current_content).strip()
                })
                id_counter += 1
            current_title = match.group(2).strip()
            current_content = []
            continue

        # Format 2: Başlık gibi görünüyor (ama sayı yok)
        elif is_probable_title(text):
            if current_title and current_content:
                fıkralar.append({
                    "ID": id_counter,
                    "Başlık": current_title,
                    "İçerik": " ".join(current_content).strip()
                })
                id_counter += 1
            current_title = text
            current_content = []
            continue

        # Normal içerik satırı
        current_content.append(text)

    # Son fıkrayı da ekle
    if current_title and current_content:
        fıkralar.append({
            "ID": id_counter,
            "Başlık": current_title,
            "İçerik": " ".join(current_content).strip()
        })

    return fıkralar

# Ana işlem
docx_path = "fikralar2.docx"
output_csv = "fikralar_duzgun2.csv"

fıkralar = parse_fikralar(docx_path)
df = pd.DataFrame(fıkralar)[["ID", "Başlık", "İçerik"]]
df.to_csv(output_csv, index=False)

print(f"{len(df)} fıkra başarıyla '{output_csv}' dosyasına kaydedildi.")

273 fıkra başarıyla 'fikralar_duzgun2.csv' dosyasına kaydedildi.


In [11]:
#